# My Q cheat sheet

## Data types

- __GUID__: The difference between using a positive integer vs. a negative integer to generate a list of GUIDs is that the positive case uses the same initial seed in each new q session whereas the negative case uses a random seed. The former is useful for reproducible results during testing but only the latter should be used in production; otherwise, your "GUIDs" will not be unique across q sessions.
- __Time__: extract millisecond and nanosecond constituents by converting them to integers and take their modulo:

In [1]:
(`int$12:34:56.789) mod 1000 / casting milliseconds
(`long$12:34:56.123456789) mod 1000000000 / nanoseconds

789


123456789


## Execution control: no iffy and loopy code

- Use boolean instead of conditionals:

In [1]:
flag:1b
base:100
base+flag*42

142


## Check if infinit value

In [ ]:
0w in (-0W; 0W) // throws error 'type

In [ ]:
myVar:0w
(myVar ~ 0W) or (myVar ~ -0W) // works

#### Create empty list

In [ ]:
0#0 ~ `long$()
0#0.0 ~ `float$()

## File I/O

- Binary file handling of q data structures as content
    - x set y:
        - x is a binary file handle including its name
        - y is a q data structure
        - saves y as x
        - file name can be customized
    - get x:
        - x is a binary file handle
        - reads x into memory as a q data structure
    - save x (similar to set[x;y]);
        - x is a filename as a symbol
        - x (ignoring path and extension) must match the name of a global table
        - saves the global table to file and returns the filename
        - the format used depends on the file extension: __.txt, .csv, .xls, .xml__
        - file name cannot be customized
        - e.g.: _save hsym `global_table_name_
        - to customize file name use this idiom:
            - ``file_name.csv 0: csv 0: table
            - saves table to .csv file while controlling the filename
            - path cannot be customized: file is saved in HOME directory

    - load x (similar to get x):
        - x is a filehandle
        - reads x and creates q data structure persisted in the file with the same name as the input filename
- Binary file handling of bites as content
    - 1: x
        - x is raw binary data 
        - writes x to binary file
        - file name cannot be customized
    - read1 x:
        - x is a binary file handle
        - reads contents of x into memory as a list of bites

- Text file handling: .txt, .csv, .tsv
    - 0: x
        - x is a list of strings
        - writes x into text file format (txt, csv, tsv)
        - path cannot be customized
    - read0 x:
        - x is a text file handle
        - reads contents of x into memory as a list of strings
        
- File handling related built-on functions:
    - hcount x: Returns as a long integer equal to the size in bytes of x
    - hsym x:
        - x is symbol containing a path ending in a file name
        - hsym convers x into a file handle by inserting a colon between the backtick and path
        - e.g.: hsym `fdf/df.txt -> `:fdf/df.txt
        - hdel x: deletes file by handle name
    - hopen x:
        - x is a file or a websocket handle
        - returns an open handle function (creates function if it does not exist)
    - hclose x: closes the file or websocket handle
    - neg h l:
        - h is an open file handle function
        - l is a list of strings
        - appends items of l to h
        - e.g.: neg h (value1;value2)

## Communication: IPC, HTTP, WebSocket

In [ ]:
.z.w (`functionName;param1;param2) // client connection handle can be used to create a callback function to the client on server side

- IPC synchronous and asynchronous request processing:

In [ ]:
.z.pg:{value x} // IPC, processes sync get requests from client q processes
.z.pg:{$[-11h=type first x; .[value first x; 1_x; ::]; `unsupported]} // example definition of .z.pg processing only functions defined on server sidea

In [ ]:
.z.ps:{value x} // IPC, processes async set requests
.z.ps:{res:$[-11h=type x[1]; .[value x[1]; 2_x; ::]; `unsupported];(neg .z.w) (value first x; res)} // async function with callback function to client

- .z.po:{[connHandle] `requestor set connHandle; <logic>;} // used to build a dictionary of connection handles and their IP addresses and user IDs
- .z.a: IP address
- .z.u: user ID
- .z.pw:{[user;pswd]1b} user id and password check

- HTTP requests

In [ ]:
.z.ph (requestText;requestHeaderAsDictionary) // for trapping HTTP get requests
.z.ph:{show x 0; show x 1; ; string value 1_x 0} // example customization on server
.z.pp:`unaryFunction // for trapping HTTP post requests, no default handler
// you can customize both of them.

In [ ]:
response:.j.j (enlist `mykey)!(enlist `myvalue);
.z.pp:{:.h.hy [`json;response]}

In [ ]:
response:enlist[`node_key]!enlist["this_is_a_node_secret"]
.z.pp:{:.h.hy [`txt;.j.j response]}

- More .z.pp implementation examples:
    - https://github.com/jonathonmcmurray/qwebapi
    - https://codywu2010.wordpress.com/2020/01/28/all-possible-ways-you-can-talk-to-q-kdb/
    - https://jmcmurray.co.uk/kdb/q/rest/api/2018/05/22/rest-api-in-kdb.html

- Websockets

- .z.ws:{neg[.z.w]x} default definition: echoes the request back to the client
- -8! to bytes
- -9! from bytes
- neg[.z.w]: sends answer to client ansychronously

In [ ]:
.z.ws:{0N!-9!x; neg[.z.w] -8!42} // for trapping web socket requests

- Pushing messages from the server

In [ ]:
\p 4242
answer:42
.z.po:{`requestor set x; system "t 1000";}
.z.ts:{neg[requestor] -8!answer;; answer+:1;}

- see also:
    - .z.wo:{[connHandle]} // evaluated when a websocket connection is initialized to a kdb session and after validated agains -u/-U file and .z.pw checks
    - .z.wc:{} // evaluated after a websocket connection is closed. used to clean up the dictionary created by .z.wo

## Idiotic one character functions

- Ternary / conditional expression:
    - $[cond_expr;true_expr;false_expr]
- Exception handling (protected evaluation)
    - @[monadic_function;single_parameter;'ErrorMessage]
    - .[multivalent_function;list_of_parameters;'ErrorMessage]
- Functional select:
    - ?[table;constraint;byGroup;aggregates;limit;orderBy] // select and exec when rank is 4 or above
    - ?[x;y;z] // vector conditional when rank is 3
    - ![] // delete and update

## The seven overloads of ?

In [ ]:
egAtom:12
egAtom2:6
egList:10+2*til 10

In [ ]:
egList?egAtom / find: returns index of atom if it is in list, else list item count

In [ ]:
egAtom?egList / roll: returns a random k (egAtom) combination of the list with repetition (n can be atom)

In [ ]:
(neg egAtom2)?egList / deal: returns a random k combination of n __without repetition__ (n>=k; n can be atom)

In [ ]:
0N?egList / permute: returns a random permutation of n

In [ ]:
egEnum:`a`b / create enum explicitly
`egEnum?`c / Enum Extend: adds item to enum object
egEnum

## String manupulation

- Join return values of system commands with raze

In [ ]:
(raze system "pwd"),"/my/path/myFile.q"

In [ ]:
.h.ty